In [ ]:
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()

exercice_id = 4
filtered_beneficiaries_pathfile = os.environ['BENEF_2025_RGPD_DUPLICATES_FILTERED_OUTPUT_PATHFILE']
output_pathfile = os.environ['BENEF_2025_TO_MIGRATE_OUTPUT_PATHFILE']

df_2025 = pd.read_csv(filtered_beneficiaries_pathfile)

In [ ]:
import random
import string
import datetime

# Code generation
current_date = datetime.datetime.now()
current_year = str(current_date.year)[-2:]

def get_characters_set(size = 4):
    return ''.join(random.choices([c for c in string.ascii_uppercase if c not in 'OI'], k=size))

def generate_code():
    return f"{current_year}-{get_characters_set(4)}-{get_characters_set(4)}"

def generate_unique_codes(n):
    unique_codes = set()
    while len(unique_codes) < n:
        code = generate_code()
        unique_codes.add(code)
    return list(unique_codes)

codes = generate_unique_codes(len(df_2025.index))

In [ ]:
assert(len(codes) == len(df_2025))

In [ ]:
import numpy as np

# Drop the codes from previous year and primary keys
df_2025.drop(columns=['id', 'allocataire-courriel'], inplace=True)

# Assign the new codes
df_2025['id_psp'] = codes

In [ ]:
# Default fields
now = pd.Timestamp.now(tz='Europe/Paris')

# We do not update the updated_at to keep track benef updates from sources coming from data providers during the current year (2025)
df_2025["created_at"] = now
df_2025['exercice_id'] = exercice_id
df_2025['uuid_doc'] = np.NaN

In [ ]:
assert(len(df_2025[df_2025['id_psp'].isna()]) == 0)

In [ ]:
df_2025.to_csv(output_pathfile, index=False)